In [1]:
import numpy as np
import sklearn.model_selection
import sklearn.preprocessing
import tensorflow as tf

In [ ]:
#method 1
#completely shuffle data, no structure from paper's setup (i.e. complete timeseries, or their tr/te splits) included

In [2]:
d_a = np.genfromtxt('../../data/21_cm/8_params_21.txt')
x = d_a[:,:-1]
y = d_a[:,-1]
print x.shape, y.shape

(3214360, 8) (3214360,)


In [3]:
m = (21562 + 2073) * 136
# print m
# print x.shape[0] #check these are same
num_outputs = 1
training_size = 21562. / (21562 + 2073) #same test proportion as in paper
test_size = 1 - training_size
train_num = int(m * training_size)
test_num = int(m * test_size) #using ceil here gives numbers inconsistent with m
cv_size = 0.1 #if vanilla cv is being used. whether to use ceil or not for cv_num depends on this value
training_size = 1 - cv_size
cv_num = int(cv_size * train_num)
train_num = int(training_size * train_num)
#note sk learn splitter uses proportion values not numbers, these are just for checks
print m
print train_num
print cv_num
print test_num
print train_num + cv_num + test_num

3214360
2639188
293243
281928
3214359


In [4]:
seed_n = 1
np.random.seed(seed_n)
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size = test_size)
y_test = y_test.reshape(-1,1)
x_train, x_cv, y_train, y_cv = sklearn.model_selection.train_test_split(x_train, y_train, test_size = cv_size)
y_train = y_train.reshape(-1,1)
y_cv = y_cv.reshape(-1,1)

#scale data to have zero mean and unit variance
x_scaler = sklearn.preprocessing.StandardScaler(copy = False)
x_scaler.fit(x_train)
x_scaler.transform(x_train)
x_scaler.transform(x_cv)
x_scaler.transform(x_test)
y_scaler = sklearn.preprocessing.StandardScaler(copy = False)
y_scaler.fit(y_train)
y_scaler.transform(y_train)
y_scaler.transform(y_cv)
y_scaler.transform(y_test)

np.savetxt("../../data/21_cm/8_params_21_x_tr.csv", x_train, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_y_tr.csv", y_train, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_x_cv.csv", x_cv, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_y_cv.csv", y_cv, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_x_te.csv", x_test, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_y_te.csv", y_test, delimiter=",")

###########################################################################
print y_scaler.mean_
#write y scaler mean here. Needed to inverse transform y_test and y_pred when evaluating 21cm performance metric
# y_scaler.mean_= -50.67502318
print y_scaler.var_
#write y scaler variance here. Needed to inverse transform y_test and y_pred when evaluating 21cm performance metric
# y_scaler.var_ = 5432.93031454
###########################################################################

[16.921533]
[-50.67502318]
[5432.93031454]


In [5]:
#method 2
#load in paper's training and test sets separately, 
#so that we know full timeseries are included in each split 
#(as they shuffle over timeseries, not individual datapoints)
#to be used for author's split of data where in the training data alpha = 1 for all the records,
#nu_min = 0.2 for all the records,
#and in the testing data alpha = 1.5 for all the records,
#nu_min = 1. for all the records.
#this causes standard scaler to screw up, so just set these values back to their original
#values after transforming

In [12]:
d_train = np.genfromtxt('../../data/21_cm/8_params_21_2_tr.txt')
x_train = d_train[:,:-1]
y_train = d_train[:,-1]
y_train = y_train.reshape(-1,1)
print x_train.shape, y_train.shape
d_test = np.genfromtxt('../../data/21_cm/8_params_21_2_te.txt')
x_test = d_test[:,:-1]
y_test = d_test[:,-1]
y_test = y_test.reshape(-1,1)
print x_test.shape, y_test.shape
##########################################
#this value needs to be copied from reconfigure.ipynb in bnn/data/.../
cv_size = 0.
##########################################
if cv_size:
    d_cv = np.genfromtxt('../../data/21_cm/8_params_21_2_cv.txt')
    x_cv = d_cv[:,:-1]
    y_cv = d_cv[:,-1]
    y_cv = y_cv.reshape(-1,1)
    print x_cv.shape, y_cv.shape

#scale data to have zero mean and unit variance
x_scaler = sklearn.preprocessing.StandardScaler(copy = False)
x_scaler.fit(x_train)
x_scaler.transform(x_train)
##########################################
# set screwed up values back to their originals
x_train[:,-4] = 1.
x_train[:,-3] = 0.2
##########################################
if cv_size:
    x_scaler.transform(x_cv)
    ##########################################
    # set screwed up values back to their originals
    # assumes cv taken from author's training split
    x_cv[:,-4] = 1.
    x_cv[:,-3] = 0.2
    ##########################################
x_scaler.transform(x_test)
##########################################
# set screwed up values back to their originals
x_test[:,-4] = 1.5
x_test[:,-3] = 1.
##########################################
y_scaler = sklearn.preprocessing.StandardScaler(copy = False)
y_scaler.fit(y_train)
y_scaler.transform(y_train)
if cv_size:
    y_scaler.transform(y_cv)
y_scaler.transform(y_test)

np.savetxt("../../data/21_cm/8_params_21_2_x_tr.csv", x_train, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_2_y_tr.csv", y_train, delimiter=",")
if cv_size:
    np.savetxt("../../data/21_cm/8_params_21_2_x_cv.csv", x_cv, delimiter=",")
    np.savetxt("../../data/21_cm/8_params_21_2_y_cv.csv", y_cv, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_2_x_te.csv", x_test, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_2_y_te.csv", y_test, delimiter=",")

###########################################################################
print y_scaler.mean_
#write y scaler mean here. Needed to inverse transform y_test and y_pred when evaluating 21cm performance metric
# y_scaler.mean_= -50.35104929
print y_scaler.var_
#write y scaler variance here. Needed to inverse transform y_test and y_pred when evaluating 21cm performance metric
# y_scaler.var_ = 5410.76986054
###########################################################################

(2932432, 8) (2932432, 1)
(281928, 8) (281928, 1)
[-50.35104929]
[5410.76986054]


In [16]:
#method 3
#load in paper's training and test sets separately, 
#so that we know full timeseries are included in each split 
#(as they shuffle over timeseries, not individual datapoints)

In [17]:
d_train = np.genfromtxt('../../data/21_cm/8_params_21_3_tr.txt')
x_train = d_train[:,:-1]
y_train = d_train[:,-1]
y_train = y_train.reshape(-1,1)
print x_train.shape, y_train.shape
d_test = np.genfromtxt('../../data/21_cm/8_params_21_3_te.txt')
x_test = d_test[:,:-1]
y_test = d_test[:,-1]
y_test = y_test.reshape(-1,1)
print x_test.shape, y_test.shape
##########################################
#this value needs to be copied from reconfigure.ipynb in bnn/data/.../
cv_size = 0.
##########################################
if cv_size:
    d_cv = np.genfromtxt('../../data/21_cm/8_params_21_3_cv.txt')
    x_cv = d_cv[:,:-1]
    y_cv = d_cv[:,-1]
    y_cv = y_cv.reshape(-1,1)
    print x_cv.shape, y_cv.shape

#scale data to have zero mean and unit variance
x_scaler = sklearn.preprocessing.StandardScaler(copy = False)
x_scaler.fit(x_train)
x_scaler.transform(x_train)
if cv_size:
    x_scaler.transform(x_cv)
x_scaler.transform(x_test)
y_scaler = sklearn.preprocessing.StandardScaler(copy = False)
y_scaler.fit(y_train)
y_scaler.transform(y_train)
if cv_size:
    y_scaler.transform(y_cv)
y_scaler.transform(y_test)

np.savetxt("../../data/21_cm/8_params_21_3_x_tr.csv", x_train, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_3_y_tr.csv", y_train, delimiter=",")
if cv_size:
    np.savetxt("../../data/21_cm/8_params_21_3_x_cv.csv", x_cv, delimiter=",")
    np.savetxt("../../data/21_cm/8_params_21_3_y_cv.csv", y_cv, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_3_x_te.csv", x_test, delimiter=",")
np.savetxt("../../data/21_cm/8_params_21_3_y_te.csv", y_test, delimiter=",")

###########################################################################
print y_scaler.mean_
#write y scaler mean here. Needed to inverse transform y_test and y_pred when evaluating 21cm performance metric
# y_scaler.mean_= -50.55857295
print y_scaler.var_
#write y scaler variance here. Needed to inverse transform y_test and y_pred when evaluating 21cm performance metric
# y_scaler.var_ = 5420.38820099
###########################################################################

(2932432, 8) (2932432, 1)
(281928, 8) (281928, 1)
[-50.55857295]
[5420.38820099]


In [18]:
x_train.mean(axis=0)

array([-3.50572872e-13, -9.41901770e-12, -6.36971423e-13, -7.32944474e-13,
        1.41901881e-13,  4.24394008e-11,  6.93835864e-15, -3.18025223e-21])